In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forecast-data/forecast_complete_data.csv
/kaggle/input/items-orders/Orders data.xlsx
/kaggle/input/items-orders/Associated_Item_data.csv


In [2]:
result = pd.read_csv(r'/kaggle/input/forecast-data/forecast_complete_data.csv')

In [3]:
result.shape

(22212949, 22)

In [4]:
result = result.iloc[:,1:]
result

,customer_id,order_item_id,order_date,quantity,lag_1_day,lag_7_days,lag_14_days,lag_30_days,rolling_mean_7d,rolling_sum_30d,...,is_std_imputed,discount_percentage,holiday,is_weekend,Friday,Monday,Saturday,Sunday,Thursday,Tuesday
0,5980,202967,2024-09-30,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,0,1,0,0,0,0
1,5980,202967,2024-10-01,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,0,0,0,0,0,1
2,5980,202967,2024-10-02,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,1,0,0,0,0,0,0,0
3,5980,202967,2024-10-03,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,0,0,0,0,1,0
4,5980,202967,2024-10-04,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22212944,7056,262487,2025-04-06,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,1,0,0,0,1,0,0
22212945,7056,262487,2025-04-07,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,0,1,0,0,0,0
22212946,7056,262487,2025-04-08,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,0,0,0,0,0,1
22212947,7056,262487,2025-04-09,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.000000,0,0,0,0,0,0,0,0


In [5]:
result['discount_percentage'].value_counts()

discount_percentage
0.000000    22102872
0.066667         481
0.111111         467
0.090909         362
0.071429         327
              ...   
0.119054           1
0.115132           1
3.579677           1
1.524164           1
0.192432           1
Name: count, Length: 26392, dtype: int64

In [6]:
X_train = result[result['order_date'] < '2025-03-01'].iloc[:,4:]
X_test = result[result['order_date'] >= '2025-03-01'].iloc[:,4:]

In [7]:
y_train = result[result['order_date'] < '2025-03-01'].iloc[:,3]
y_test = result[result['order_date'] >= '2025-03-01'].iloc[:,3]

In [8]:
# For gradient boosting approach
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [9]:
model = LGBMRegressor(n_estimators=100, learning_rate=0.05)
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.725752 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 740
[LightGBM] [Info] Number of data points in the train set: 17494136, number of used features: 17
[LightGBM] [Info] Start training from score 0.034840


LGBMRegressor(learning_rate=0.05)

In [10]:
preds = model.predict(X_test)
preds

array([4.10740469e-04, 4.10740469e-04, 4.10740469e-04, ...,
       4.10740469e-04, 4.10740469e-04, 1.76423423e+00])

In [11]:
pred_quantity = pd.DataFrame(np.round(preds))
pred_quantity.value_counts()

0     
 0.0      4694316
 1.0         8854
 2.0         4008
 5.0         1931
 3.0         1814
           ...   
 510.0          1
 556.0          1
 611.0          1
-287.0          1
 733.0          1
Name: count, Length: 166, dtype: int64

In [12]:
mae = mean_absolute_error(y_test, preds)
mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)

# Adjusted R2 calculation
n = len(y_test)          # number of samples
p = X_test.shape[1]      # number of features (independent variables)
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)

print(f'Test MAE: {mae}')
print(f'Test MSE: {mse}')
print(f'Test R2: {r2}')
print(f'Test Adjusted R2: {adjusted_r2}')

Test MAE: 0.007874720625264845
Test MSE: 0.6046870313062066
Test R2: 0.7080882290755617
Test Adjusted R2: 0.7080871774299391
